![static_arm_0deg](static_arm_0deg.png)<br>
![static_arm_22deg](static_arm_22deg.png)<br>
![static_arm_45deg](static_arm_45deg.png)

In [25]:

import math

force = 120  # The applied force in Newtons
angle_degrees = 22  # Angle of the blue member with the horizontal
force = 100  # The applied force in Newtons
angle_radians = math.radians(angle_degrees)

f_x = force * math.cos(angle_radians)
f_y = force * math.sin(angle_radians)

f_x_result = -f_x / 2
f_y_result = -f_y / 2

print("Resultant x force: ", f_x_result)
print("Resultant y force: ", f_y_result)

Resultant x force:  -46.35919272833937
Resultant y force:  -18.7303296707956


![](https://blog.igus.in/wp-content/uploads/2021/10/efficiencycalculate.jpg)

In [26]:
def calculate_lead_screw_efficiency(lead, diameter, coefficient_of_friction, input_unit='imperial'):
    """
    Calculate the efficiency of a lead screw considering different materials for screw and nut.

    :param lead: The lead of the screw (distance the nut moves per screw revolution).
    :param diameter: The nominal diameter of the screw.
    :param coefficient_of_friction: The coefficient of friction between screw and nut materials.
    :param input_unit: 'imperial' or 'metric'.
    :return: Efficiency of the lead screw.
    """
    # Convert all inputs to imperial units for calculation
    if input_unit == 'metric':
        lead = lead * 0.0393701  # Convert lead to inches
        diameter = diameter * 0.0393701  # Convert diameter to inches

    # Calculate pitch diameter, assuming it's close to the nominal diameter
    pitch_diameter = diameter

    # Calculate the lead angle
    lead_angle = math.atan(lead / (math.pi * pitch_diameter))

    # Calculate angle of friction
    angle_of_friction = math.atan(coefficient_of_friction)

    # Calculate efficiency
    efficiency = math.tan(lead_angle) / math.tan(lead_angle + angle_of_friction)

    return efficiency  # Fractional efficiency

def calculate_pitch(threads_per_pitch, input_unit='imperial', output_unit='imperial'):
    """
    Calculate the pitch of a lead screw.

    :param threads_per_pitch: Threads per inch if imperial, pitch in mm if metric.
    :param input_unit: 'imperial' or 'metric'.
    :param output_unit: 'imperial' (threads per inch) or 'metric' (pitch in mm).
    :return: Pitch of the lead screw in the desired unit.
    """
    if input_unit == 'imperial':
        if output_unit == 'metric':
            # Convert threads per inch to pitch in mm (1 inch = 25.4 mm)
            pitch = 25.4 / threads_per_pitch
        else:
            pitch = threads_per_pitch  # Already in threads per inch
    else:  # input is metric
        if output_unit == 'imperial':
            # Convert pitch in mm to threads per inch (1 inch = 25.4 mm)
            pitch = 25.4 / threads_per_pitch
        else:
            pitch = threads_per_pitch  # Already in mm

    return pitch

# From : https://www.mcmaster.com/95075A340/ (PET Plastic Externally Threaded Precision Acme Nut)
coefficient_of_friction = 0.22  # Estimated for steel on PET plastic
# from: https://www.mcmaster.com/99030A985/ (1018 Carbon Steel Precision Acme Lead Screw)
lead = 0.05 # imperial
diameter = 0.25 # imperial
threads_per_pitch = 20 # imperial

lead_screw_pitch = calculate_pitch(threads_per_pitch, input_unit='imperial', output_unit='metric')
print("Lead screw pitch: ", lead_screw_pitch)
lead_screw_efficiency = calculate_lead_screw_efficiency(lead, diameter, coefficient_of_friction, 'imperial')
print("Lead screw efficiency: ", lead_screw_efficiency)

Lead screw pitch:  1.27
Lead screw efficiency:  0.2212857409800622


![Lead Screw Nut Torque](https://dev.nookindustries.com/media/3266/5ds.png)

In [27]:
def calculate_driving_torque(load, lead, efficiency):
    """
    Calculate the driving torque for a screw.
    
    Parameters:
    load (float): The load in pounds (lb).
    lead (float): The screw lead in inches/turn.
    efficiency (float): The efficiency of the screw.
    
    Returns:
    float: The driving torque in pound-inches.
    """
    return (load * lead) / (2 * math.pi * efficiency)

def calculate_holding_torque(load, lead, efficiency):
    """
    Calculate the holding torque for a screw.
    
    Parameters:
    load (float): The load in pounds (lb).
    lead (float): The screw lead in inches/turn.
    efficiency (float): The efficiency of the screw.
    
    Returns:
    float: The holding torque in pound-inches.
    """
    return (load * lead * efficiency) / (2 * math.pi)

def pound_inches_to_kg_cm(pound_inches):
    """
    Convert pound-inches to kilogram-centimeters.
    
    Parameters:
    pound_inches (float): The torque in pound-inches.
    
    Returns:
    float: The torque in kilogram-centimeters.
    """
    conversion_factor = 0.86796165979664
    return pound_inches * conversion_factor

def kg_cm_to_pound_inches(kg_cm):
    """
    Convert kilogram-centimeters to pound-inches.
    
    Parameters:
    kg_cm (float): The torque in kilogram-centimeters.
    
    Returns:
    float: The torque in pound-inches.
    """
    conversion_factor = 0.86796165979664
    return kg_cm / conversion_factor

# Example usage:
load = abs(f_y_result) * 3  # Example load in pounds
load_lbs = load / 4.4482216

# Calculate torques
driving_torque = calculate_driving_torque(load_lbs, lead, lead_screw_efficiency)
holding_torque = calculate_holding_torque(load_lbs, lead, lead_screw_efficiency)
print(pound_inches_to_kg_cm(driving_torque), "kg-cm")
print(pound_inches_to_kg_cm(holding_torque), "kg-cm")

0.39429158549885757 kg-cm
0.01930742556715194 kg-cm
